In [1]:
import csv
import json
import sqlite3

results = {'inlab':[], 'inlab_gender':[], 'inlab_nogender':[], 'mturk':[], 'machine':[], 'combined':[]}
results = {'mturk':[]}
results = {'AIexperts':[]}

############# for jason file processing
jsonpath = './Data_all/'
#filenames = {'inlab': ['turing_test_img_caption_inlab_gender_15','turing_test_img_caption_inlab_no_gender_15'],
#             'inlab_gender': ['turing_test_img_caption_inlab_gender_15'],
#             'inlab_nogender': ['turing_test_img_caption_inlab_no_gender_15'],
#             'mturk':['turing_live']}
filenames = {'AIexperts':['turing_live_may4']}

#Images 1-250 are from COCO test set, 
# images 251-500 are nocaps val in-domain, 
# images 501-750 are nocaps val near-domain, 
# images 751-1000 are nocaps val out-domain


In [2]:
json.loads(all_data[2][17])['data'][2]

NameError: name 'all_data' is not defined

In [ ]:
len(trials_dict.keys())

In [3]:
for key in filenames:
    print("=======================")
    print(key)

    count = 0
    trials_dict = {}
    workercount = 0
    
    for dbfilename in filenames[key]:
        #print("=======================")
        #print(jasonfilename)
        conn = sqlite3.connect(jsonpath + dbfilename + ".db")
        cursor = conn.cursor()
        cursor.execute("select * from imgcaption;")        
        all_data = cursor.fetchall()

        for i in range(0, len(all_data)):
            if (all_data[i][15] == 3): #Elisa: in sandbox, this is 3   or (all_data[i][15] == 5): #check status 
                data_dict = json.loads(all_data[i][17])
                workercount += 1
                for dat in data_dict['data']:
                    if dat['trialdata']['phase'] == 'TEST':
                        
                        count += 1
                        trial_data = {}
                        trial_data['workerID'] = data_dict['workerId']
                        trial_data['workerData'] = data_dict['questiondata']
                        trial_data['trialData'] = dat['trialdata']
                        
                        imagepathname_id = int(trial_data['trialData']['imageID'][-8:-4])
                    
                        if imagepathname_id<= 250:
                            trial_data['workerData']['imagetype'] = 'COCO'
                        elif imagepathname_id<= 500:
                            trial_data['workerData']['imagetype'] = 'in-domain'
                        elif imagepathname_id<= 750:
                            trial_data['workerData']['imagetype'] = 'near-domain'
                        else:
                            trial_data['workerData']['imagetype'] = 'out-domain'
                        
                        
                        # Get rid of typo in the key in db file
                        if 'ai_grountruth' in list(trial_data['trialData'].keys()):
                            trial_data['trialData']['machine_groundtruth'] = trial_data['trialData']['ai_grountruth']
                            del trial_data['trialData']['ai_grountruth']

                        # Get rid of typo in the key in db file
                        if trial_data['trialData']['response_speaker'] == 'ai':
                            trial_data['trialData']['response_speaker'] = 'machine'

                        trials_dict['Trial_'+str(count)] = trial_data

                
        conn.close()
        results[key] = trials_dict
        #json.dump(trials_dict, open("./turing_inlab_no_gender_img_caption_half.json","w"))
    print(workercount, ' workers')

########### AI jason files
# json_file_path = './Data_all/ai_judge_curie_all_seed.json'

# with open(json_file_path, 'r') as j:
#      contents = json.loads(j.read())
# results['machine'] = contents
# #print(contents)

# #save combined results
# count = 0
# trials_dict = {}
# for ke in ['mturk']:  #ke in ['inlab','mturk']:
#     for it in results[ke]:
#         count = count + 1
#         trialin = results[ke][it]
#         trials_dict['Trial_'+str(count)] = trialin
        
# results['combined'] = trials_dict

# print('All data is extracted.')
# for key in results:
#     print(key)
#     print('There are [' + str(len(results[key])) + ']')
    
#save the compiled results as jason
with open("./savedResults_elisa/compiled_AIexperts_may4.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

AIexperts
11  workers
Compilation completed! Results are saved to folder!


In [4]:
results['mturk']['Trial_1']

KeyError: 'mturk'

In [ ]:
results['inlab_gender']['Trial_1']